## Install necessary modules

In [4]:
#pip install -q pandas==1.1.5
#pip install ipython-sql

## Import relevant modules and connect to a new database

In [5]:
import csv, sqlite3, pandas as pd
 
con = sqlite3.connect("Customer_Shopping_DB.db")
cur = con.cursor()

## Load the SQL extension in Jupyter Notebooks

In [6]:
%load_ext sql

## Establish a connection between SQL magic module and the database "Customer_Shopping_DB.db"

In [7]:
%sql sqlite:///Customer_Shopping_DB.db

## Load CSV file into a dataframe 

In [8]:
customer_shopping_data = "customer_shopping_data.csv"
df = pd.read_csv(customer_shopping_data)
df

,invoice_no,customer_id,gender,age,category,quantity,price,payment_method,invoice_date,shopping_mall
0,I138884,C241288,Female,28,Clothing,5,1500.40,Credit Card,5/8/2022,Kanyon
1,I317333,C111565,Male,21,Shoes,3,1800.51,Debit Card,12/12/2021,Forum Istanbul
2,I127801,C266599,Male,20,Clothing,1,300.08,Cash,9/11/2021,Metrocity
3,I173702,C988172,Female,66,Shoes,5,3000.85,Credit Card,16/05/2021,Metropol AVM
4,I337046,C189076,Female,53,Books,4,60.60,Cash,24/10/2021,Kanyon
...,...,...,...,...,...,...,...,...,...,...
99452,I219422,C441542,Female,45,Souvenir,5,58.65,Credit Card,21/09/2022,Kanyon
99453,I325143,C569580,Male,27,Food & Beverage,2,10.46,Cash,22/09/2021,Forum Istanbul
99454,I824010,C103292,Male,63,Food & Beverage,2,10.46,Debit Card,28/03/2021,Metrocity
99455,I702964,C800631,Male,56,Technology,4,4200.00,Cash,16/03/2021,Istinye Park


## Check for missing values

In [9]:
missing_values = df.isnull().sum()
print("Missing values in each column:")
display(missing_values)

Missing values in each column:


invoice_no        0
customer_id       0
gender            0
age               0
category          0
quantity          0
price             0
payment_method    0
invoice_date      0
shopping_mall     0
dtype: int64

## Check for duplicates

In [10]:
duplicate_rows = df[df.duplicated()]
print("Duplicate rows:")
display(duplicate_rows)

Duplicate rows:


,invoice_no,customer_id,gender,age,category,quantity,price,payment_method,invoice_date,shopping_mall


## Data formatting

In [11]:
df['price'] = df['price'].round(2) #Change "price" to 2 decimal place
df['invoice_date'] = pd.to_datetime(df['invoice_date'], format='%d/%m/%Y') #Transform "invoice_date" to date format
df

,invoice_no,customer_id,gender,age,category,quantity,price,payment_method,invoice_date,shopping_mall
0,I138884,C241288,Female,28,Clothing,5,1500.40,Credit Card,2022-08-05,Kanyon
1,I317333,C111565,Male,21,Shoes,3,1800.51,Debit Card,2021-12-12,Forum Istanbul
2,I127801,C266599,Male,20,Clothing,1,300.08,Cash,2021-11-09,Metrocity
3,I173702,C988172,Female,66,Shoes,5,3000.85,Credit Card,2021-05-16,Metropol AVM
4,I337046,C189076,Female,53,Books,4,60.60,Cash,2021-10-24,Kanyon
...,...,...,...,...,...,...,...,...,...,...
99452,I219422,C441542,Female,45,Souvenir,5,58.65,Credit Card,2022-09-21,Kanyon
99453,I325143,C569580,Male,27,Food & Beverage,2,10.46,Cash,2021-09-22,Forum Istanbul
99454,I824010,C103292,Male,63,Food & Beverage,2,10.46,Debit Card,2021-03-28,Metrocity
99455,I702964,C800631,Male,56,Technology,4,4200.00,Cash,2021-03-16,Istinye Park


## Input cleaned dataframe as a table "CUSTOMER_SHOPPING_DATA" into the database 

In [12]:
df.to_sql("CUSTOMER_SHOPPING_DATA", con, if_exists='replace', index=False,method="multi", chunksize=50)

## Check if the table is created

In [13]:
%sql SELECT name FROM sqlite_master WHERE type='table'

 * sqlite:///Customer_Shopping_DB.db
Done.


name
CUSTOMER_SHOPPING_DATA


## Check if the number of rows tallies with the CSV file

In [14]:
%sql SELECT COUNT(*) FROM CUSTOMER_SHOPPING_DATA;

 * sqlite:///Customer_Shopping_DB.db
Done.


COUNT(*)
99457


## Display top 5 rows of the table as a preview

In [15]:
%sql SELECT * FROM CUSTOMER_SHOPPING_DATA LIMIT 5;

 * sqlite:///Customer_Shopping_DB.db
Done.


invoice_no,customer_id,gender,age,category,quantity,price,payment_method,invoice_date,shopping_mall
I138884,C241288,Female,28,Clothing,5,1500.4,Credit Card,2022-08-05 00:00:00,Kanyon
I317333,C111565,Male,21,Shoes,3,1800.51,Debit Card,2021-12-12 00:00:00,Forum Istanbul
I127801,C266599,Male,20,Clothing,1,300.08,Cash,2021-11-09 00:00:00,Metrocity
I173702,C988172,Female,66,Shoes,5,3000.85,Credit Card,2021-05-16 00:00:00,Metropol AVM
I337046,C189076,Female,53,Books,4,60.6,Cash,2021-10-24 00:00:00,Kanyon


# Questions to Consider
## Q1: What is the total number of invoices and sales based on gender?

In [16]:
%%sql SELECT gender, COUNT(*) AS total_invoices, ROUND(sum(price), 2) AS total_sales FROM CUSTOMER_SHOPPING_DATA 
GROUP BY gender;

 * sqlite:///Customer_Shopping_DB.db
Done.


gender,total_invoices,total_sales
Female,59482,40931801.62
Male,39975,27619564.29


## Q2: What is the total number of invoices and sales based on age range? Rank them from highest to lowest sales.

In [17]:
%%sql
SELECT 
    CASE
        WHEN age BETWEEN 0 AND 19 THEN '0-19'
        WHEN age BETWEEN 20 AND 29 THEN '20-29'
        WHEN age BETWEEN 30 AND 39 THEN '30-39'
        WHEN age BETWEEN 40 AND 49 THEN '40-49'
        WHEN age BETWEEN 50 AND 59 THEN '50-59'
        WHEN age BETWEEN 60 AND 69 THEN '60-69'
        WHEN age BETWEEN 70 AND 79 THEN '70-79'
        WHEN age BETWEEN 80 AND 89 THEN '80-89'
        WHEN age >= 90 THEN '90+'
    END AS age_range, 
    COUNT(*) AS total_invoices, 
    ROUND(SUM(price), 2) AS total_sales 
FROM CUSTOMER_SHOPPING_DATA 
GROUP BY age_range
ORDER BY total_sales DESC;

 * sqlite:///Customer_Shopping_DB.db
Done.


age_range,total_invoices,total_sales
40-49,19153,13376514.35
20-29,19263,13324658.44
30-39,19287,13245827.96
60-69,19043,13160725.05
50-59,18931,12937020.02
0-19,3780,2506620.09


## Q3: What is the total number of invoices and sales by payment method?

In [18]:
%%sql SELECT payment_method, COUNT(*) AS total_invoices, ROUND(sum(price), 2) AS total_sales FROM CUSTOMER_SHOPPING_DATA 
GROUP BY payment_method;

 * sqlite:///Customer_Shopping_DB.db
Done.


payment_method,total_invoices,total_sales
Cash,44447,30705030.98
Credit Card,34931,24051476.93
Debit Card,20079,13794858.0


## Q4: What is the average sales for all product categories?

In [20]:
%%sql 
SELECT category, ROUND(avg(price),2) AS average_sales FROM CUSTOMER_SHOPPING_DATA 
GROUP BY category 
ORDER BY average_sales DESC;

 * sqlite:///Customer_Shopping_DB.db
Done.


category,average_sales
Technology,3156.94
Shoes,1807.39
Clothing,901.08
Cosmetics,122.45
Toys,107.73
Books,45.57
Souvenir,34.89
Food & Beverage,15.67


## Q5: What is the top 3 most popular product categories and their total sales?

In [21]:
%%sql 
SELECT category, ROUND(sum(price), 2) AS total_sales FROM CUSTOMER_SHOPPING_DATA 
GROUP BY category
ORDER BY sum(price)
DESC LIMIT 3;

 * sqlite:///Customer_Shopping_DB.db
Done.


category,total_sales
Clothing,31075684.64
Shoes,18135336.89
Technology,15772050.0


## Q6: Which shopping mall contributed to the lowest sales?

In [22]:
%%sql 
SELECT shopping_mall, ROUND(sum(price), 2) AS total_sales FROM CUSTOMER_SHOPPING_DATA
GROUP BY shopping_mall
ORDER BY sum(price)
ASC LIMIT 1;

 * sqlite:///Customer_Shopping_DB.db
Done.


shopping_mall,total_sales
Forum Istanbul,3336073.82


## Q7: Categorize the sales transactions into months and find out which month contributed the most sales in 2022.

In [23]:
%%sql 
SELECT strftime('%m-%Y', invoice_date) AS 'month', COUNT(*) AS total_invoices, ROUND(sum(price), 2) AS total_sales
FROM CUSTOMER_SHOPPING_DATA
GROUP BY month
ORDER BY strftime('%Y', invoice_date);

 * sqlite:///Customer_Shopping_DB.db
Done.


month,total_invoices,total_sales
01-2021,3835,2656422.78
02-2021,3407,2358636.34
03-2021,3813,2618434.14
04-2021,3724,2558825.62
05-2021,3848,2662369.93
06-2021,3783,2547239.73
07-2021,3984,2802468.58
08-2021,3723,2632303.32
09-2021,3670,2530305.88
10-2021,3916,2782418.4


In [24]:
%%sql 
SELECT month, total_invoices, "total sales" 
FROM (
SELECT strftime('%m-%Y', invoice_date) AS month, COUNT(*) AS total_invoices, ROUND(sum(price), 2) AS "total sales"
FROM CUSTOMER_SHOPPING_DATA
GROUP BY month
HAVING month LIKE '%2022'
ORDER BY strftime('%Y', invoice_date)
)
ORDER BY "total sales" DESC
LIMIT 1;

 * sqlite:///Customer_Shopping_DB.db
Done.


month,total_invoices,total sales
10-2022,3848,2755839.69


In [25]:
#Close Connection
con.close()